About the DataSet:

1.Id:Unique id for a news article

2.title:the title of a news article

3.author: author of the news article

4.text: the text of the article ;could be compltete

5.label: a label that marks whether the news article is real or fake

     1: Fake news

      0: real news

In [16]:
import numpy as np
import pandas as pd
import re
import sklearn
import pickle
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import nltk

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
#printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-Processing

In [27]:
#Loading the dataset to a pandas Dataframes
news_dataset = pd.read_csv('/content/train.csv')

ParserError: Error tokenizing data. C error: Expected 5 fields in line 196, saw 7


In [26]:
news_dataset.shape

NameError: name 'news_dataset' is not defined

In [ ]:
news_dataset.head()

In [ ]:
#counting the number of missing values in the dataset
print("\nMissing values:")
print(news_dataset.isnull().sum())

In [ ]:
#replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [ ]:
#Merging the author name and news title
news_dataset['content'] = news_dataset['author'] + ' ' + news_dataset['title']
print("\nCombined content:")
print(news_dataset['content'])

In [ ]:
#separating the data and label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']
print("\nFeatures:")
print(X)
print("\nLabels:")
print(Y)

Stemming:

Stemming is the process of reducing a word to its root word

example:

actor,actress,acting-->act

In [ ]:
port_stem = PorterStemmer()

def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:
# Apply stemming to content
news_dataset['content'] = news_dataset['content'].apply(stemming)
print("\nProcessed content:")
print(news_dataset['content'])

In [ ]:
# Separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values
print("\nFeature array:")
print(X)
print("\nLabel array:")
print(Y)
print("\nLabel shape:", Y.shape)

In [ ]:
# Converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)
print("\nVectorized features:")
print(X)

Spliting the dataSet to trainig and test data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

Training the model Logistic Regression

In [ ]:
model=LogisticRegression();

In [ ]:
model.fit(X_train,Y_train)

Evaluation

 Accuracy Score

In [ ]:
# Accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print('\nAccuracy score of the training data:', training_data_accuracy)

In [ ]:
# Accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy score of the test data:', test_data_accuracy)

Making a Predictive System

In [ ]:
X_new = X_test[0]
prediction = model.predict(X_new)
if prediction[0] == 0:
    print("\nThe news is Real")
else:
    print("\nThe news is Fake")

In [ ]:
# Print version information
print("\nNumpy version:", np.__version__)
print("Pandas version:", pd.__version__)
print("Scikit-learn version:", sklearn.__version__)

Saving the trained model and vectorizer together

In [ ]:
model_data = {
    'model': model,
    'vectorizer': vectorizer
}
with open('trained_model.pkl', 'wb') as file:
    pickle.dump(model_data, file)

print("\nModel and vectorizer saved successfully!")